In [2]:

import os

current_path = os.getcwd()
dataset_path = os.path.join(current_path,"../dataset")
    
novelSet_path = os.path.join(dataset_path, "visual-novels")
save_novelJson_path = os.path.join(dataset_path, "visual-novels-yaml-v2")

print("novelSet_path: ", novelSet_path)

novel_list = os.listdir(novelSet_path)

novelSet_path:  c:\Users\kimbo\code\insight_text_game\novel_dataset\../dataset\visual-novels


In [3]:
def create_folder(_path):
    if not os.path.exists(_path):
        os.makedirs(_path, exist_ok=True)

In [4]:
create_folder(dataset_path)
create_folder(save_novelJson_path)

In [ ]:
test_dict = dict('test')
def test (value):
    print(value)

test(test_dict)

In [5]:
class dict(object):
    def __init__(self, name):
        self._name = name

    def __str__(self):
        return self._name

    def __repr__(self):
        return "'"+str(self._name)+"'"

class Gmark:
    def __init__(self):
        self.sig=False
        self.cnt=0
        self.idx=0
        self.json_dict={}
        self.novel_title=""
    
    def add_cnt(self):
        self.cnt+=1
    def add_idx(self):
        self.idx+=1
        
    def get_cnt(self):
        return self.cnt
    def get_idx(self):
        return self.idx

    def reset_cnt(self):
        self.cnt=0
    def reset_idx(self):
        self.idx=0

    def set_sig_true(self):
        self.sig=True
    def set_sig_false(self):
        self.sig=False
    def get_sig(self):
        return self.sig
    
    def reset_json_dict(self):
        self.json_dict.clear()
    def add_json_dict(self, name, content):
        self.json_dict[dict(name)]=content
    def get_json_dict(self):
        return self.json_dict
    def get_json_dict_len(self):
        return len(self.json_dict)
    
    def set_novel_title(self, title):
        self.novel_title=title
    def get_novel_title(self):
        return self.novel_title

In [6]:
def qeustion_marker(name, content, gmark):
    if gmark.get_cnt()>3:
        if gmark.get_sig() & ('?' in content):
            gmark.set_sig_false()
            
        if gmark.get_sig():
            gmark.set_sig_false()
            gmark.reset_cnt()
            gmark.add_json_dict('question', "===============================")
            gmark.add_json_dict(name,content)
            gmark.add_json_dict('answer', "===============================")
            return
        
    if '?' in content:
        gmark.add_json_dict(name, content)
        gmark.add_cnt()
        gmark.set_sig_true()
    else:
        gmark.add_json_dict(name, content)
        gmark.set_sig_false()
        gmark.add_cnt()

In [7]:
class TrainData():
    def __init__(self):
        self.trainSet=[]
        self.input_novel={}
        self.answer={}
        self.output_novel={}
        self.train={}
        self.q=False
        self.a=False
        
    def set_q(self, boolean):
        self.q=boolean
    def set_a(self, boolean):
        self.a=boolean
        
    def get_q(self):
        return self.q
    def get_a(self):
        return self.a
    
    def add_value(self, tag, name, content):
        if tag =="input_novel":
            self.input_novel[dict(name)] = content
        elif tag =="answer":
            self.answer[dict(name)] = content
        elif tag =="output_novel":
            self.output_novel[dict(name)] = content
        elif tag == "train":
            self.train[name] = content.copy()
        elif tag == "trainSet":
            self.trainSet.append(content.copy())
        else:
            print("No such key exist.")
    
    def reset_value(self, name):
        if name == "input_novel":
            self.input_novel.clear()
        elif name == "answer":
            self.answer.clear()
        elif name == "output_novel":
            self.output_novel.clear()
        elif name == "train":
            self.train.clear()
            
    def get_value(self, name):
        if name =="input_novel":
            return self.input_novel
        elif name =="answer":
            return self.answer
        elif name =="output_novel":
            return self.output_novel
        elif name =="train":
            return self.train
        elif name =="trainSet":
            return self.trainSet
        else:
            print("No such key exist.")
            
    def change_trainSet(self):
        self.input_novel = self.output_novel.copy()
        self.reset_value('answer')
        self.reset_value('output_novel')
        self.reset_value('train')

In [22]:
def conversion_to_trainData(gmark):
    def save_value():
        train_data.add_value('train', 'input_novel', train_data.get_value('input_novel'))
        train_data .add_value('train', 'answer', train_data.get_value('answer'))
        train_data.add_value('train', 'output_novel', train_data.get_value('output_novel'))
        train_data.add_value('trainSet', 'train', train_data.get_value('train'))
    
    qmark_data = gmark.get_json_dict()
    q=0
    train_data = TrainData()
    for name, value in qmark_data.items():
        if(str(name) == "question"):
            train_data.set_q(True)
            q+=1
            if(q!=1):
                save_value()
                
                train_data.change_trainSet()
                train_data.set_a(False)
            continue
        if(str(name) == "answer"):
            train_data.set_a(True)
            continue
                
        if not train_data.get_q() and not train_data.get_a():
            train_data.add_value('input_novel', name, value)
        elif train_data.get_q() and not train_data.get_a():
            train_data.add_value('answer', name, value)
        elif train_data.get_q() and train_data.get_a():
            train_data.add_value('output_novel', name, value)
        else:
            print("Error: No matching question or answer found.")
    
    if train_data.get_value('input_novel'):
        save_value()
    return train_data.get_value('trainSet')

In [9]:

'''
['Shikkoku no Sharnoth ~What a beautiful tomorrow~', 'Euphoria'] 제외
['A Clockwork Ley-Line - The Borderline of Dusk'] name이 To의 경우 narraition으로 처리
['YOU', 'You', '<USER>']의 경우 'user'로 통일
나레이션 태그 없는 경우도 나레이션으로 처리
나레이션의 ( )는 strip으로 제거
['Girls Frontline (Extras)', 'Girls Frontline','MonsterGirlQuest'] 제외
['Rewrite']는 ['*snip*', '*chink*'] 제거
['Sisterly Bliss (Tsui Yuri ~Okaa-san ni wa Naisho da yo~)']의 Narrator 태그는 narraition으로 처리
['The Eden of Grisaia', 'The Fruit of Grisaia', 'The Labyrinth of Grisaia']의 Narration 태그는 narraition으로 처리
'''
case_1 = ['Shikkoku no Sharnoth ~What a beautiful tomorrow~', 'Euphoria']
case_2 = ['A Clockwork Ley-Line - The Borderline of Dusk']
case_3 = ['Girls Frontline (Extras)', 'Girls Frontline','MonsterGirlQuest']
case_4 = ['Rewrite']
case_5 = ['Sisterly Bliss (Tsui Yuri ~Okaa-san ni wa Naisho da yo~)']
case_6 = ['The Eden of Grisaia', 'The Fruit of Grisaia', 'The Labyrinth of Grisaia']
case_7 = ['Dawn Of Malice', 'Love At First Sight', 'Academy34', 'Dawn Of Malice', 'Doki Doki Literature Club', 'Prince Of Suburbia', 'REFLEXIA Prototype ver', 'Sugar Style']

In [19]:
import yaml

def exception(gmark, novel_content):
    title = gmark.get_novel_title()
    if title in case_1:
        return
    elif title in case_2:
        case_2_func(gmark, novel_content)
    elif title in case_3:
        return
    elif title in case_4:
        case_4_func(gmark, novel_content)
    elif title in case_5:
        case_56_func(gmark, novel_content)
    elif title in case_6:
        case_56_func(gmark, novel_content)
    else:
        save(gmark, novel_content)
      
def save_novel(gmark):
    gmark.add_idx()
    train_set = conversion_to_trainData(gmark)
    save_to_yaml(gmark.get_idx(), gmark.get_novel_title(), train_set)
    gmark.reset_json_dict()
    
def case_2_func(gmark, novel_contents):
    for sentence in novel_contents:
        try:
            if 'To' in sentence:
                name, content = sentence.split(":", 1)
                qeustion_marker('narration', content.strip()[1:-1], gmark)
            elif sentence[0] == "*":
                qeustion_marker('narration', sentence.strip()[1:-1], gmark)
            elif sentence[0] == "=":
                save_novel(gmark)
            else:
                name, content = sentence.split(":", 1)
                qeustion_marker(name.strip(), content.strip(), gmark)
        except:
            qeustion_marker('unknown', sentence.strip(), gmark)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
        
def case_4_func(gmark, novel_contents):
    for sentence in novel_contents:
        try:
            if "*snip*" in sentence or "*chink*" in sentence:
                continue
            elif sentence[0] == "*":
                qeustion_marker('narration', sentence.strip()[1:-1], gmark)
            elif sentence[0] == "=":
                save_novel(gmark)
            else:
                name, content = sentence.split(":", 1)
                qeustion_marker(name.strip(), content.strip(), gmark)
        except:
            qeustion_marker('unknown', sentence.strip(), gmark)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
        
def case_56_func(gmark, novel_contents):
    for sentence in novel_contents:
        try:
            if 'Narration' in novel_contents or 'Narrator' in novel_contents:
                qeustion_marker('narration', sentence.strip()[1:-1], gmark)
            elif sentence[0] == "*":
                qeustion_marker('narration', sentence.strip()[1:-1], gmark)
            elif sentence[0] == "=":
                save_novel(gmark)
            else:
                name, content = sentence.split(":", 1)
                qeustion_marker(name.strip(), content.strip(), gmark)
        except:
            qeustion_marker('unknown', sentence.strip(), gmark)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
        
def save(gmark, novel_contents):
    for sentence in novel_contents:
        try:
            if sentence[0] == "*":
                qeustion_marker('narration', sentence.strip()[1:-1], gmark)
            elif sentence[0] == "=":
                save_novel(gmark)
            else:
                name, content = sentence.split(":", 1)
                qeustion_marker(name.strip(), content.strip(), gmark)
        except:
            qeustion_marker('unknown', sentence.strip(), gmark)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
        
def save_to_yaml(idx, novel_title, result):
    idx = str(idx)
    file_name = novel_title+"_"+str(idx)+".yaml"
    save_json_path = os.path.join(save_novelJson_path, novel_title, file_name)
    
    with open(save_json_path, 'w', encoding='utf-8') as f:
        yaml.dump(result, f, indent=4)

In [ ]:
import yaml

def exception(gmark, novel_content):
    title = gmark.get_novel_title()
    if title in case_1:
        return
    elif title in case_2:
        case_2_func(gmark, novel_content)
    elif title in case_3:
        return
    elif title in case_4:
        case_4_func(gmark, novel_content)
    elif title in case_5:
        case_56_func(gmark, novel_content)
    elif title in case_6:
        case_56_func(gmark, novel_content)
    elif title in case_7:
        case_7_func(gmark, novel_content)
    else:
        save(gmark, novel_content)
      
def save_novel(gmark):
    gmark.add_idx()
    train_set = conversion_to_trainData(gmark)
    save_to_yaml(gmark.get_idx(), gmark.get_novel_title(), train_set)
    gmark.reset_json_dict()
    
def case_2_func(gmark, novel_contents):
    for sentence in novel_contents:
        if 'To' in sentence:
            name, content = sentence.split(":", 1)
            qeustion_marker('narration', content.strip()[1:-1], gmark)
        same_save_logic(gmark, sentence)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
        
def case_4_func(gmark, novel_contents):
    for sentence in novel_contents:
        if "*snip*" in sentence or "*chink*" in sentence:
            continue
        same_save_logic(gmark, sentence)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
        
def case_56_func(gmark, novel_contents):
    for sentence in novel_contents:
        if 'Narration' in novel_contents or 'Narrator' in novel_contents:
            qeustion_marker('narration', sentence.strip()[1:-1], gmark)
        same_save_logic(gmark, sentence)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
        
def case_7_func(gmark, novel_contents):
    for sentence in novel_contents:
        if 'YOU' in novel_contents or 'You' in novel_contents or '<USER>' in novel_contents:
            qeustion_marker('narration', sentence.strip()[1:-1], gmark)
        same_save_logic(gmark, sentence)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
def save(gmark, novel_contents):
    for sentence in novel_contents:
        same_save_logic(gmark, sentence)
    
    if(gmark.get_json_dict_len()!=0):
        save_novel(gmark)
        
def same_save_logic(gmark, sentence):
    try:
        if sentence[0] == "*":
            qeustion_marker('narration', sentence.strip()[1:-1], gmark)
        elif sentence[0] == "=":
            save_novel(gmark)
        else:
            name, content = sentence.split(":", 1)
            qeustion_marker(name.strip(), content.strip(), gmark)
    except:
        qeustion_marker('unknown', sentence.strip(), gmark)
        
def save_to_yaml(idx, novel_title, result):
    idx = str(idx)
    file_name = novel_title+"_"+str(idx)+".yaml"
    save_json_path = os.path.join(save_novelJson_path, novel_title, file_name)
    
    with open(save_json_path, 'w', encoding='utf-8') as f:
        yaml.dump(result, f, indent=4)

In [11]:
def novel_convert_to_json(novel_title, novel_contents):
    gmark = Gmark()
    
    novel_title = novel_title.strip('.txt')
    save_path = os.path.join(save_novelJson_path, novel_title)
    create_folder(save_path)
    gmark.set_novel_title(novel_title)
    
    exception(gmark, novel_contents)

In [23]:

for novel_name in novel_list:
    open_file = os.path.join(novelSet_path, novel_name)
    f = open(open_file, 'r', encoding='utf-8')
    novel_content = []
    while True:
        line = f.readline()
        if not line: 
            break
        novel_content.append(line.strip())
    f.close()
    novel_convert_to_json(novel_name, novel_content)